# 5. Resampling Methods – Applied

Excercises from **Chapter 5** of [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/) by Gareth James, Daniela Witten, Trevor Hastie and Robert Tibshirani.

I've elected to use Python instead of R.

In [107]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from IPython.display import display, HTML
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [60]:
def confusion_table(confusion_mtx):
    """Renders a nice confusion table with labels"""
    confusion_df = pd.DataFrame({'y_pred=0': np.append(confusion_mtx[:, 0], confusion_mtx.sum(axis=0)[0]),
                                 'y_pred=1': np.append(confusion_mtx[:, 1], confusion_mtx.sum(axis=0)[1]),
                                 'Total': np.append(confusion_mtx.sum(axis=1), ''),
                                 '': ['y=0', 'y=1', 'Total']}).set_index('')
    return confusion_df

def total_error_rate(confusion_matrix):
    """Derive total error rate from confusion matrix"""
    return 1 - np.trace(confusion_mtx) / np.sum(confusion_mtx)

### 5. In Chapter 4, we used logistic regression to predict the probability of default using income and balance on the Default data set. We will now estimate the test error of this logistic regression model using the validation set approach. Do not forget to set a random seed before beginning your analysis.

In [129]:
default_df = pd.read_csv('./data/Default.csv', index_col='Unnamed: 0')
default_df = default_df.reset_index().drop('index', axis=1)

# Check for missing
assert default_df.isna().sum().sum() == 0

# Rationalise types
default_df = pd.get_dummies(default_df, dtype=np.float64).drop(['default_No', 'student_No'], axis=1)

display(default_df.head())

,balance,income,default_Yes,student_Yes
0,729.526495,44361.625074,0.0,0.0
1,817.180407,12106.134700,0.0,1.0
2,1073.549164,31767.138947,0.0,0.0
3,529.250605,35704.493935,0.0,0.0
4,785.655883,38463.495879,0.0,0.0


### (a) Fit a logistic regression model that uses income and balance to predict default.
### (b) Using the validation set approach, estimate the test error of this model. In order to do this, you must perform the following steps:

- i. Split the sample set into a training set and a validation set.
- ii. Fit a multiple logistic regression model using only the training observations.
- iii. Obtain a prediction of default status for each individual in the validation set by computing the posterior probability of default for that individual, and classifying the individual to the default category if the posterior probability is greater than 0.5.
- iv. Compute the validation set error, which is the fraction of the observations in the validation set that are misclassified.

### (c) Repeat the process in (b) three times, using three different splits of the observations into a training set and a validation set. Com- ment on the results obtained.

In [130]:
for s in range(1,4):
    display(HTML('<h3>Random seed = {}</h3>'.format(s)))
    # Create index for 50% holdout set
    np.random.seed(s)
    train = np.random.rand(len(default_df)) < 0.5
    
    response   = 'default_Yes'
    predictors = ['income', 'balance']
    
    X_train = np.array(default_df[train][predictors])
    X_test  = np.array(default_df[~train][predictors])
    y_train = np.array(default_df[train][response])
    y_test  = np.array(default_df[~train][response])
    
    # Logistic regression
    logit       = LogisticRegression()
    model_logit = logit.fit(X_train, y_train)
    
    # Predict
    y_pred = model_logit.predict(X_test)
    
    # Analysis
    confusion_mtx = confusion_matrix(y_test, y_pred)
    display(confusion_table(confusion_mtx))
    
    total_error_rate_pct = np.around(total_error_rate(confusion_mtx) * 100, 4)
    print('total_error_rate: {}%'.format(total_error_rate_pct))


,y_pred=0,y_pred=1,Total
,,,
y=0,4846,0,4846
y=1,164,0,164
Total,5010,0,


total_error_rate: 3.2735%


,y_pred=0,y_pred=1,Total
,,,
y=0,4691,1,4692
y=1,176,0,176
Total,4867,1,


total_error_rate: 3.636%


,y_pred=0,y_pred=1,Total
,,,
y=0,4773,1,4774
y=1,163,0,163
Total,4936,1,


total_error_rate: 3.3219%


### (d) Now consider a logistic regression model that predicts the probability of default using income, balance, and a dummy variable for student. Estimate the test error for this model using the validation set approach. Comment on whether or not including a dummy variable for student leads to a reduction in the test error rate.

In [131]:
for s in range(1,4):
    display(HTML('<h3>Random seed = {}</h3>'.format(s)))
    # Create index for 50% holdout set
    np.random.seed(s)
    train = np.random.rand(len(default_df)) < 0.5
    
    response   = 'default_Yes'
    predictors = ['income', 'balance', 'student_Yes']
    
    X_train = np.array(default_df[train][predictors])
    X_test  = np.array(default_df[~train][predictors])
    y_train = np.array(default_df[train][response])
    y_test  = np.array(default_df[~train][response])
    
    # Logistic regression
    logit       = LogisticRegression()
    model_logit = logit.fit(X_train, y_train)
    
    # Predict
    y_pred = model_logit.predict(X_test)
    
    # Analysis
    confusion_mtx = confusion_matrix(y_test, y_pred)
    display(confusion_table(confusion_mtx))
    
    total_error_rate_pct = np.around(total_error_rate(confusion_mtx) * 100, 4)
    print('total_error_rate: {}%'.format(total_error_rate_pct))

,y_pred=0,y_pred=1,Total
,,,
y=0,4846,0,4846
y=1,164,0,164
Total,5010,0,


total_error_rate: 3.2735%


,y_pred=0,y_pred=1,Total
,,,
y=0,4688,4,4692
y=1,172,4,176
Total,4860,8,


total_error_rate: 3.6154%


,y_pred=0,y_pred=1,Total
,,,
y=0,4773,1,4774
y=1,163,0,163
Total,4936,1,


total_error_rate: 3.3219%


**Comment**

It is difficult to discern if the student predictor has improved the model because of the variation in results.


### 6. We continue to consider the use of a logistic regression model to predict the probability of default using income and balance on the Default data set. In particular, we will now compute estimates for the standard errors of the income and balance logistic regression coefficients in two different ways: (1) using the bootstrap, and (2) using the standard formula for computing the standard errors in the glm() function. Do not forget to set a random seed before beginning your analysis.

### (a) Using the summary() and glm() functions, determine the estimated standard errors for the coefficients associated with income and balance in a multiple logistic regression model that uses both predictors.

In [293]:
response   = 'default_Yes'
predictors = ['income', 'balance']

X_all = sm.add_constant(np.array(default_df[predictors]))
y_all = np.array(default_df[response])

## Logistic regression
model_logit = smf.Logit(y_all, X_all).fit(disp=False);    

# Summary
print(model_logit.summary())

statsmodels_est = pd.DataFrame({'coef_sm': model_logit.params, 'SE_sm': model_logit.bse})
display(statsmodels_est)

                           Logit Regression Results                           
Dep. Variable:                      y   No. Observations:                10000
Model:                          Logit   Df Residuals:                     9997
Method:                           MLE   Df Model:                            2
Date:                Wed, 19 Sep 2018   Pseudo R-squ.:                  0.4594
Time:                        22:15:14   Log-Likelihood:                -789.48
converged:                       True   LL-Null:                       -1460.3
                                        LLR p-value:                4.541e-292
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        -11.5405      0.435    -26.544      0.000     -12.393     -10.688
x1          2.081e-05   4.99e-06      4.174      0.000     1.1e-05    3.06e-05
x2             0.0056      0.000     24.835      0.0

,coef_sm,SE_sm
0,-11.540468,0.434772
1,0.000021,0.000005
2,0.005647,0.000227


**NOTE:** Apparent bug in statsmodels.discrete.discrete_model.LogitResults.summary. Std error x2 is misrepresented in summary table, it is easy to misread this as lower than standard error for x1 when in fact it is not (see table above).

### (b) Write a function, boot.fn(), that takes as input the Default data set as well as an index of the observations, and that outputs the coefficient estimates for income and balance in the multiple logistic regression model.

In [272]:
def boot_fn(df, idx):
    response   = 'default_Yes'
    predictors = ['income', 'balance']
    
    X = sm.add_constant(np.array(df[predictors].loc[idx]));
    y = np.array(df[response].loc[idx]) 
       
    # Logistic regression
    model_logit = smf.Logit(y, X).fit(disp=False);  
    return model_logit.params;


### (c) Use the boot() function together with your boot.fn() function to estimate the standard errors of the logistic regression coefficients for income and balance.

In [300]:
from scipy import stats

def boot_idx(n):
    """Return index for bootstrap sample of size n
    e.g. generate array in range 0 to n, with replacement"""
    return np.random.randint(low=0, high=n, size=n)

def boot(fn, data_df, samples):
    """Perform bootstrap for B number of samples"""
    results = []
    for s in range(samples):
        Z = fn(data_df, boot_idx(data_df.shape[0]))
        results += [Z]
    return np.array(results)

def standard_error(X):
    """Compute standard error for jth element in matrix X
    equivalent to np.std(X, axis=0)"""
    X_bar = np.mean(X, axis=0)
    SE = np.sqrt((np.sum(np.square(X - X_bar), axis=0)) / (len(X)))
    return SE

B = 10000
coef_preds    = boot(boot_fn, default_df, samples=B)
coef_pred     = np.mean(coef_preds, axis=0)
standard_errs = standard_error(coef_preds)

bootstrap_est = pd.DataFrame({'coef_boot': coef_pred, 'SE_boot': standard_errs})
display(bootstrap_est)


,coef_boot,SE_boot
0,-11.569426,0.431264
1,0.000021,0.000005
2,0.005664,0.000227


### (d) Comment on the estimated standard errors obtained using the glm() function and using your bootstrap function.

In [301]:
pd.concat([statsmodels_est, bootstrap_est], axis=1)


,coef_sm,SE_sm,coef_boot,SE_boot
0,-11.540468,0.434772,-11.569426,0.431264
1,0.000021,0.000005,0.000021,0.000005
2,0.005647,0.000227,0.005664,0.000227


Let's compare the standard errors estimated by the statsmodels (_sm) summary() function with estimates obtained by bootstrap (_boot) in the table above. The standard errors for x1 and x2 (rows 1 and 2) are indistinguishable to 6 decimal places. The coefficient for x2 and the statistics for the intercept x0 vary slightly.

Note that the disparity is slightly more significant when fewer bootstrap samples are used. Here 10,000 were used, but the estimates were alike to within the same order of magnitude with only 10 bootstrap samples.